In [18]:
import iris

import numpy as np
import scipy.stats

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[0]

cm_hist = iris.load_cube("testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc")
cm_future = iris.load_cube("testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc")

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [21]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [22]:
# Get data
obs = np.array(obs.data[:, 0, :, :])
cm_hist = cm_hist.data
cm_future = cm_future.data

Test ISIMIP:

In [23]:
from isimip import ISIMIP

temp_debiaser = ISIMIP(variable = "tas")

In [24]:
obs = obs[:, 1, 1]
cm_hist = cm_hist[:, 1, 1]
cm_future = cm_future[:, 1, 1]

In [25]:
temp_debiaser.step5(obs, cm_hist, cm_future)

IndexError: index 432 is out of bounds for axis 0 with size 432

In [11]:
from switanek2017 import Switanek2017
temp_debiaser = Switanek2017()
debiased_cm5 = temp_debiaser.absolute_sdm_location(obs[:,1,1], cm_hist[:,1,1], cm_future[:,1,1])

In [12]:
debiased_cm4-debiased_cm5

masked_array(data=[561.1606146967908, 620.9720802263178,
                   528.4494059040615, 507.51448391259953,
                   286.24350013101594, 226.47546302322257,
                   519.216175969339, 504.85053038784656,
                   398.08813378712887, 388.9076985268727,
                   611.6611879105994, 574.8041831352879,
                   731.9400575651226, 610.8525026704112, 532.451663773567,
                   532.4325396570428, 445.82052673763366,
                   466.5710558926103, 135.52900221796563,
                   228.95845162649402, 172.28739122473922,
                   316.81877140666643, 566.539154966927,
                   514.4426552476037, 1527.2428933125457,
                   458.0565998563618, 526.7478147240492,
                   547.5320652215295, 586.907999718904,
                   207.70759814968906, 24.10298253875419,
                   -15.561038509597893, 262.719525957899,
                   566.4768578964043, 520.4981178623873, 448